# GeoHarness v4.0: Local/CPU Scikit-Learn Offset Decoder Training

이 노트북은 `GeoHarness` 프로젝트에서 생성한 `ml_dataset.csv` (100건 내외)를 기반으로, Google WGS84 좌표계와 Naver KATECH/TM128 좌표계 사이의 **보안 오프셋(Encryption Offset)**을 리버스 엔지니어링하는 머신러닝 모델을 학습합니다.

데이터 크기가 작으므로 무거운 GPU / RAPIDS / XGBoost 환경을 구축할 필요 없이, **로컬 CPU (Scikit-Learn Random Forest Regressor)** 만으로 수 초 내에 학습을 완료할 수 있습니다.

### 1. 환경 설정 및 라이브러리 설치
VS Code나 로컬 환경에서 필요한 라이브러리(`scikit-learn`, `pandas`, `pyproj`)가 없다면 아래 셀을 실행하여 설치하세요.

In [ ]:
!pip install scikit-learn pandas pyproj

### 2. 데이터 로딩
프로젝트 `data` 경로에 있는 `ml_dataset.csv`를 읽어옵니다. (코랩인 경우 좌측 패널에 업로드 필요)

In [ ]:
import os
import pandas as pd
from pathlib import Path

# 현재 노트북 위치를 기준으로 프로젝트 루트(GeoHarness) 탐색
current_dir = Path.cwd()
while current_dir.name != 'GeoHarness' and current_dir.parent != current_dir:
    current_dir = current_dir.parent

# 로컬 환경 (VS Code) 경로 매칭 (필요시 경로 수정)
dataset_path = current_dir / "data" / "ml_dataset.csv"
if not dataset_path.exists():
    dataset_path = Path("ml_dataset.csv") # 코랩 기준 (업로드된 파일)

try:
    df = pd.read_csv(dataset_path)
    print(f"Data loaded: {len(df)} rows from {dataset_path}")
except FileNotFoundError:
    import urllib.request
    print("File not found. Please upload ml_dataset.csv to current directory or fix the path.")
df.head()

### 3. 데이터 전처리 (Features & Targets 설정)
- **X (입력값):** WGS84 좌표계의 Google `Lng(경도)`, `Lat(위도)`
- **Y (목표값):** 네이버에서 반환하는 고유 KATECH 좌표(m단위) `n_mapx`, `n_mapy`

In [ ]:
import numpy as np

X = df[['g_lng', 'g_lat']].values  # X: WGS84

# 타겟: WGS84에서 예상되는 좌표와 실제 Naver 좌표의 차이
y_x = df['n_mapx'].values
y_y = df['n_mapy'].values

print("Feature shape:", X.shape)
print("Target shape:", y_x.shape)

### 4. Scikit-Learn Random Forest 모델 학습
Random Forest 회귀 알고리즘을 사용하여 비선형적 공간 왜곡 트리를 계산합니다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, yx_train, yx_test, yy_train, yy_test = train_test_split(
    X, y_x, y_y, test_size=0.1, random_state=42
)

model_x = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    random_state=42,
    n_jobs=-1 # 모든 논리 프로세서 사용
)

model_y = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    random_state=42,
    n_jobs=-1
)

print("Training X/Lng KATECH projection model...")
model_x.fit(X_train, yx_train)

print("Training Y/Lat KATECH projection model...")
model_y.fit(X_train, yy_train)
print("Training Complete!")

### 5. 정확도 평가 및 모델 추출 (`.pkl`)

In [ ]:
from sklearn.metrics import mean_squared_error
import pickle

pred_x = model_x.predict(X_test)
pred_y = model_y.predict(X_test)

rmse_x = np.sqrt(mean_squared_error(yx_test, pred_x))
rmse_y = np.sqrt(mean_squared_error(yy_test, pred_y))

print(f"RMSE X (KATECH units ≈ meters): {rmse_x:.2f} m")
print(f"RMSE Y (KATECH units ≈ meters): {rmse_y:.2f} m")
print(f"Average Error: {(rmse_x + rmse_y)/2:.2f} m")

# 모델 직렬화 (Pickle) - GeoHarness에서 디코더로 불러옵니다.
# 모델 저장 위치를 프로젝트 루트/models 폴더로 지정
model_dir = current_dir / "models"
model_dir.mkdir(exist_ok=True)

model_filename = model_dir / 'sklearn_offset_model.pkl'
try:
    with open(model_filename, 'wb') as f:
        pickle.dump({'model_x': model_x, 'model_y': model_y}, f)
    print(f"Model saved locally as {model_filename}")
except Exception as e:
    print(f"Failed to save model to {model_filename}: {e}")
    with open('sklearn_offset_model.pkl', 'wb') as f:
        pickle.dump({'model_x': model_x, 'model_y': model_y}, f)
    print("Model saved in current directory instead.")
    try:
        from google.colab import files
        files.download('sklearn_offset_model.pkl')
    except:
        pass